In [1]:
n = 1000
male = 0.5
alczheimer_chance = 0.3

In [2]:
import pandas as pd
unpruned_rules = pd.read_csv("data.csv", sep=';')
unpruned_rules

,num,rule,prediction,fr_X,fr_XC,generated
0,1,smoking,AD,300,125,False
1,2,stress,AD,500,150,False
2,3,higheducation,~AD,500,400,False
3,4,tea,~AD,342,240,False
4,5,turmeric,~AD,2,2,False
5,6,female,~AD,500,352,False
6,7,"female,stress",AD,260,100,False
7,8,"berries,apples",AD,120,32,False
8,9,"smoking,tea",AD,240,100,False
9,10,"smoking,higheducation",AD,80,32,False


In [3]:
unpruned_rules['p_X'] = unpruned_rules['fr_X'] / n
unpruned_rules["p_C"] = unpruned_rules['prediction'].map({'AD': alczheimer_chance, '~AD': (
        1 - alczheimer_chance)})
unpruned_rules['p_not_X'] = 1 - unpruned_rules['p_X']
unpruned_rules['p_not_C'] = 1 - unpruned_rules['p_C']

unpruned_rules['p_XC'] = unpruned_rules['fr_XC'] / n
unpruned_rules['p_X_not_C'] = unpruned_rules['p_X'] - unpruned_rules['p_XC']
unpruned_rules['p_not_X_C'] = unpruned_rules['p_C'] - unpruned_rules['p_XC']
unpruned_rules['p_not_X_not_C'] = unpruned_rules['p_not_X'] - unpruned_rules['p_not_X_C']
unpruned_rules

,num,rule,prediction,fr_X,fr_XC,generated,p_X,p_C,p_not_X,p_not_C,p_XC,p_X_not_C,p_not_X_C,p_not_X_not_C
0,1,smoking,AD,300,125,False,0.300,0.3,0.700,0.7,0.125,0.175,0.175,0.525
1,2,stress,AD,500,150,False,0.500,0.3,0.500,0.7,0.150,0.350,0.150,0.350
2,3,higheducation,~AD,500,400,False,0.500,0.7,0.500,0.3,0.400,0.100,0.300,0.200
3,4,tea,~AD,342,240,False,0.342,0.7,0.658,0.3,0.240,0.102,0.460,0.198
4,5,turmeric,~AD,2,2,False,0.002,0.7,0.998,0.3,0.002,0.000,0.698,0.300
5,6,female,~AD,500,352,False,0.500,0.7,0.500,0.3,0.352,0.148,0.348,0.152
6,7,"female,stress",AD,260,100,False,0.260,0.3,0.740,0.7,0.100,0.160,0.200,0.540
7,8,"berries,apples",AD,120,32,False,0.120,0.3,0.880,0.7,0.032,0.088,0.268,0.612
8,9,"smoking,tea",AD,240,100,False,0.240,0.3,0.760,0.7,0.100,0.140,0.200,0.560
9,10,"smoking,higheducation",AD,80,32,False,0.080,0.3,0.920,0.7,0.032,0.048,0.268,0.652


In [4]:
from numpy import log2
def pp(x):
    return x**x
def mi(df):
    numerator = pp(df['p_XC']) * pp(df['p_X_not_C']) * pp(df['p_not_X_C']) * pp(df['p_not_X_not_C'])
    denominator = pp(df['p_X']) * pp(df['p_not_X']) * pp(df['p_C']) * pp(df['p_not_C'])
    return log2(numerator / denominator)

def n_mi(df):
    return mi(df) * n

In [5]:
unpruned_rules["confidence"] = unpruned_rules['fr_XC']/unpruned_rules['fr_X']
unpruned_rules["leverage"] = (
        (unpruned_rules['p_XC']/n) - ((unpruned_rules['p_X']/n) * unpruned_rules["p_C"]))
unpruned_rules['lift'] = unpruned_rules['confidence'] / unpruned_rules["p_C"]
unpruned_rules['mi'] =  mi(unpruned_rules)
unpruned_rules['n_mi'] = n_mi(unpruned_rules)
unpruned_rules


,num,rule,prediction,fr_X,fr_XC,generated,p_X,p_C,p_not_X,p_not_C,p_XC,p_X_not_C,p_not_X_C,p_not_X_not_C,confidence,leverage,lift,mi,n_mi
0,1,smoking,AD,300,125,False,0.300,0.3,0.700,0.7,0.125,0.175,0.175,0.525,0.416667,3.500000e-05,1.388889,0.019436,19.435585
1,2,stress,AD,500,150,False,0.500,0.3,0.500,0.7,0.150,0.350,0.150,0.350,0.300000,0.000000e+00,1.000000,0.000000,0.000000
2,3,higheducation,~AD,500,400,False,0.500,0.7,0.500,0.3,0.400,0.100,0.300,0.200,0.800000,5.000000e-05,1.142857,0.034852,34.851555
3,4,tea,~AD,342,240,False,0.342,0.7,0.658,0.3,0.240,0.102,0.460,0.198,0.701754,6.000000e-07,1.002506,0.000005,0.005498
4,5,turmeric,~AD,2,2,False,0.002,0.7,0.998,0.3,0.002,0.000,0.698,0.300,1.000000,6.000000e-07,1.428571,0.001030,1.030385
5,6,female,~AD,500,352,False,0.500,0.7,0.500,0.3,0.352,0.148,0.348,0.152,0.704000,2.000000e-06,1.005714,0.000055,0.054961
6,7,"female,stress",AD,260,100,False,0.260,0.3,0.740,0.7,0.100,0.160,0.200,0.540,0.384615,2.200000e-05,1.282051,0.008399,8.398762
7,8,"berries,apples",AD,120,32,False,0.120,0.3,0.880,0.7,0.032,0.088,0.268,0.612,0.266667,-4.000000e-06,0.888889,0.000531,0.530725
8,9,"smoking,tea",AD,240,100,False,0.240,0.3,0.760,0.7,0.100,0.140,0.200,0.560,0.416667,2.800000e-05,1.388889,0.014202,14.201863
9,10,"smoking,higheducation",AD,80,32,False,0.080,0.3,0.920,0.7,0.032,0.048,0.268,0.652,0.400000,8.000000e-06,1.333333,0.002847,2.846664


In [6]:
removed_rules = unpruned_rules[unpruned_rules['lift'] <= 1].copy()
removed_rules[['rule', 'prediction', 'confidence', 'leverage', 'lift']]

,rule,prediction,confidence,leverage,lift
1,stress,AD,0.300000,0.000000,1.000000
7,"berries,apples",AD,0.266667,-0.000004,0.888889
12,female,AD,0.296000,-0.000002,0.986667


In [7]:
pruned_rules = unpruned_rules[unpruned_rules['lift'] > 1].copy()
pruned_rules

,num,rule,prediction,fr_X,fr_XC,generated,p_X,p_C,p_not_X,p_not_C,p_XC,p_X_not_C,p_not_X_C,p_not_X_not_C,confidence,leverage,lift,mi,n_mi
0,1,smoking,AD,300,125,False,0.300,0.3,0.700,0.7,0.125,0.175,0.175,0.525,0.416667,3.500000e-05,1.388889,0.019436,19.435585
2,3,higheducation,~AD,500,400,False,0.500,0.7,0.500,0.3,0.400,0.100,0.300,0.200,0.800000,5.000000e-05,1.142857,0.034852,34.851555
3,4,tea,~AD,342,240,False,0.342,0.7,0.658,0.3,0.240,0.102,0.460,0.198,0.701754,6.000000e-07,1.002506,0.000005,0.005498
4,5,turmeric,~AD,2,2,False,0.002,0.7,0.998,0.3,0.002,0.000,0.698,0.300,1.000000,6.000000e-07,1.428571,0.001030,1.030385
5,6,female,~AD,500,352,False,0.500,0.7,0.500,0.3,0.352,0.148,0.348,0.152,0.704000,2.000000e-06,1.005714,0.000055,0.054961
6,7,"female,stress",AD,260,100,False,0.260,0.3,0.740,0.7,0.100,0.160,0.200,0.540,0.384615,2.200000e-05,1.282051,0.008399,8.398762
8,9,"smoking,tea",AD,240,100,False,0.240,0.3,0.760,0.7,0.100,0.140,0.200,0.560,0.416667,2.800000e-05,1.388889,0.014202,14.201863
9,10,"smoking,higheducation",AD,80,32,False,0.080,0.3,0.920,0.7,0.032,0.048,0.268,0.652,0.400000,8.000000e-06,1.333333,0.002847,2.846664
10,11,"stress,smoking",AD,200,100,False,0.200,0.3,0.800,0.7,0.100,0.100,0.200,0.600,0.500000,4.000000e-05,1.666667,0.032268,32.268400
11,12,"female,higheducation",~AD,251,203,False,0.251,0.7,0.749,0.3,0.203,0.048,0.497,0.252,0.808765,2.730000e-05,1.155378,0.014462,14.461504


In [8]:

pruned_rules["mi"] =  mi(pruned_rules)
pruned_rules["n_mi"] = n_mi(pruned_rules)
pruned_rules

,num,rule,prediction,fr_X,fr_XC,generated,p_X,p_C,p_not_X,p_not_C,p_XC,p_X_not_C,p_not_X_C,p_not_X_not_C,confidence,leverage,lift,mi,n_mi
0,1,smoking,AD,300,125,False,0.300,0.3,0.700,0.7,0.125,0.175,0.175,0.525,0.416667,3.500000e-05,1.388889,0.019436,19.435585
2,3,higheducation,~AD,500,400,False,0.500,0.7,0.500,0.3,0.400,0.100,0.300,0.200,0.800000,5.000000e-05,1.142857,0.034852,34.851555
3,4,tea,~AD,342,240,False,0.342,0.7,0.658,0.3,0.240,0.102,0.460,0.198,0.701754,6.000000e-07,1.002506,0.000005,0.005498
4,5,turmeric,~AD,2,2,False,0.002,0.7,0.998,0.3,0.002,0.000,0.698,0.300,1.000000,6.000000e-07,1.428571,0.001030,1.030385
5,6,female,~AD,500,352,False,0.500,0.7,0.500,0.3,0.352,0.148,0.348,0.152,0.704000,2.000000e-06,1.005714,0.000055,0.054961
6,7,"female,stress",AD,260,100,False,0.260,0.3,0.740,0.7,0.100,0.160,0.200,0.540,0.384615,2.200000e-05,1.282051,0.008399,8.398762
8,9,"smoking,tea",AD,240,100,False,0.240,0.3,0.760,0.7,0.100,0.140,0.200,0.560,0.416667,2.800000e-05,1.388889,0.014202,14.201863
9,10,"smoking,higheducation",AD,80,32,False,0.080,0.3,0.920,0.7,0.032,0.048,0.268,0.652,0.400000,8.000000e-06,1.333333,0.002847,2.846664
10,11,"stress,smoking",AD,200,100,False,0.200,0.3,0.800,0.7,0.100,0.100,0.200,0.600,0.500000,4.000000e-05,1.666667,0.032268,32.268400
11,12,"female,higheducation",~AD,251,203,False,0.251,0.7,0.749,0.3,0.203,0.048,0.497,0.252,0.808765,2.730000e-05,1.155378,0.014462,14.461504


In [9]:
nmi_removed_rules = pruned_rules[pruned_rules['n_mi'] <= 1.5].copy()
nmi_removed_rules[['rule', 'prediction', 'confidence', 'leverage', 'lift', 'n_mi']]

,rule,prediction,confidence,leverage,lift,n_mi
3,tea,~AD,0.701754,6.000000e-07,1.002506,0.005498
4,turmeric,~AD,1.000000,6.000000e-07,1.428571,1.030385
5,female,~AD,0.704000,2.000000e-06,1.005714,0.054961


In [10]:
nmi_pruned_rules = pruned_rules[pruned_rules['n_mi'] > 1.5].copy()
nmi_pruned_rules[['rule', 'prediction', 'confidence', 'leverage', 'lift', 'n_mi']]

,rule,prediction,confidence,leverage,lift,n_mi
0,smoking,AD,0.416667,0.000035,1.388889,19.435585
2,higheducation,~AD,0.800000,0.000050,1.142857,34.851555
6,"female,stress",AD,0.384615,0.000022,1.282051,8.398762
8,"smoking,tea",AD,0.416667,0.000028,1.388889,14.201863
9,"smoking,higheducation",AD,0.400000,0.000008,1.333333,2.846664
10,"stress,smoking",AD,0.500000,0.000040,1.666667,32.268400
11,"female,higheducation",~AD,0.808765,0.000027,1.155378,14.461504


1. turmeric - 100% confidence, small number of examples, likely overfitting
2. female and stress, Valid lift, but low confidence.
3. Female and stress by themselfes alone is independent, or predict slightly the opposite, but together they have strong dependence





In [19]:
def create_removable_array_based_on_values(rule_df):
    removable_array = []
    for _, row in rule_df.iterrows():
        has_better_sub_rule = False
        basic_rules_list = row['rule'].split(',')
        if len(basic_rules_list) == 1:
            removable_array.append(False) # Keep single rules
            continue
        for basic_rule_str in basic_rules_list:
            matching_rule = unpruned_rules[
                (unpruned_rules['rule'] == basic_rule_str ) &
                (unpruned_rules['prediction'] == row['prediction'])
            ]
            if len(matching_rule) == 0: continue
            if len(matching_rule) > 1: raise Exception("More than one matching rule")

            basic_rule_confidence = matching_rule['confidence'].item()
            if  basic_rule_confidence >= row['confidence']:
                print(f"{row['rule']}: {basic_rule_str} confidence is {basic_rule_confidence}, which is better than {row['confidence']}")
                has_better_sub_rule = True
                break
        removable_array.append(has_better_sub_rule)
    print(removable_array)
    return pd.Series(removable_array, index=rule_df.index)

nmi_pruned_rules[create_removable_array_based_on_values(nmi_pruned_rules)]

smoking,tea: smoking confidence is 0.4166666666666667, which is better than 0.4166666666666667
smoking,higheducation: smoking confidence is 0.4166666666666667, which is better than 0.4
[False, False, False, True, True, False, False]


,num,rule,prediction,fr_X,fr_XC,generated,p_X,p_C,p_not_X,p_not_C,p_XC,p_X_not_C,p_not_X_C,p_not_X_not_C,confidence,leverage,lift,mi,n_mi
8,9,"smoking,tea",AD,240,100,False,0.24,0.3,0.76,0.7,0.100,0.140,0.200,0.560,0.416667,0.000028,1.388889,0.014202,14.201863
9,10,"smoking,higheducation",AD,80,32,False,0.08,0.3,0.92,0.7,0.032,0.048,0.268,0.652,0.400000,0.000008,1.333333,0.002847,2.846664


In [18]:
def mi_c(child, parent):
    p_XQ = child['p_X']
    p_XQC = child['p_XC']

    p_X = parent['p_X']
    p_XC = parent['p_XC']

    p_X_not_C = p_X - p_XC

    p_X_not_Q = p_X - p_XQ

    p_XQ_not_C = p_XQ - p_XQC
    p_X_not_Q_C = p_XC - p_XQC
    p_X_not_Q_not_C = p_X_not_Q - p_X_not_Q_C


    numerator = pp(p_X)*pp(p_XQC)*pp(p_XQ_not_C)*pp(p_X_not_Q_C)*pp(p_X_not_Q_not_C)
    denominator = pp(p_XQ)*pp(p_X_not_Q)*pp(p_XC)*pp(p_X_not_C)

    mi_c = log2(numerator / denominator)
    if mi_c < 0.0000001:
        return 0
    return mi_c
def n_mi_c(child, parent):
    return mi_c(child, parent) * n

def create_removable_array_based_on_mi(rule_df):
    removable_array = []
    for _, row in rule_df.iterrows():
        has_better_sub_rule = False
        basic_rules_list = row['rule'].split(',')
        if len(basic_rules_list) == 1:
            removable_array.append(False) # Keep single rules
            continue
        for basic_rule_str in basic_rules_list:
            matching_rule_df = unpruned_rules[
                (unpruned_rules['rule'] == basic_rule_str) &
                (unpruned_rules['prediction'] == row['prediction'])
                ]
            if len(matching_rule_df) == 0: continue
            if len(matching_rule_df) > 1: raise Exception("More than one matching rule")

            if n_mi_c(row, matching_rule_df.iloc[0]) < 0.5:
                print(f"{row['rule']}: {basic_rule_str}'s and {matching_rule_df.iloc[0]['rule']} nm_c is {n_mi_c(row, matching_rule_df.iloc[0])}")

                has_better_sub_rule = True
                break
        removable_array.append(has_better_sub_rule)
    print(removable_array)
    return pd.Series(removable_array, index=rule_df.index)

nmi_pruned_rules[create_removable_array_based_on_mi(nmi_pruned_rules)]

smoking,tea: smoking's and smoking nm_c is 0
smoking,higheducation: smoking's and smoking nm_c is 0.09016711665929368
female,higheducation: higheducation's and higheducation nm_c is 0.1745993165351095
[False, False, False, True, True, False, True]


,num,rule,prediction,fr_X,fr_XC,generated,p_X,p_C,p_not_X,p_not_C,p_XC,p_X_not_C,p_not_X_C,p_not_X_not_C,confidence,leverage,lift,mi,n_mi
8,9,"smoking,tea",AD,240,100,False,0.240,0.3,0.760,0.7,0.100,0.140,0.200,0.560,0.416667,0.000028,1.388889,0.014202,14.201863
9,10,"smoking,higheducation",AD,80,32,False,0.080,0.3,0.920,0.7,0.032,0.048,0.268,0.652,0.400000,0.000008,1.333333,0.002847,2.846664
11,12,"female,higheducation",~AD,251,203,False,0.251,0.7,0.749,0.3,0.203,0.048,0.497,0.252,0.808765,0.000027,1.155378,0.014462,14.461504


In [13]:
value_pruned = nmi_pruned_rules[~create_removable_array_based_on_values(nmi_pruned_rules)]
final = value_pruned[~create_removable_array_based_on_mi(value_pruned)]
final

smoking,tea: smoking confidence is 0.4166666666666667, which is better than 0.4166666666666667
smoking,higheducation: smoking confidence is 0.4166666666666667, which is better than 0.4
[False, False, False, True, True, False, False]
[False, False, False, False, True]


,num,rule,prediction,fr_X,fr_XC,generated,p_X,p_C,p_not_X,p_not_C,p_XC,p_X_not_C,p_not_X_C,p_not_X_not_C,confidence,leverage,lift,mi,n_mi
0,1,smoking,AD,300,125,False,0.30,0.3,0.70,0.7,0.125,0.175,0.175,0.525,0.416667,0.000035,1.388889,0.019436,19.435585
2,3,higheducation,~AD,500,400,False,0.50,0.7,0.50,0.3,0.400,0.100,0.300,0.200,0.800000,0.000050,1.142857,0.034852,34.851555
6,7,"female,stress",AD,260,100,False,0.26,0.3,0.74,0.7,0.100,0.160,0.200,0.540,0.384615,0.000022,1.282051,0.008399,8.398762
10,11,"stress,smoking",AD,200,100,False,0.20,0.3,0.80,0.7,0.100,0.100,0.200,0.600,0.500000,0.000040,1.666667,0.032268,32.268400


3.2.b Stress and somking


3.2.c Smoking and high education implies that high education particpitates in